<a href="https://colab.research.google.com/github/isaacpy-cloud/TraderWolfs/blob/main/Traders_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance
!pip install catboost
!pip install matplotlib --upgrade
!pip install mpl_finance
!pip install tradingview-ta
!pip install websocket-client

  Using cached https://files.pythonhosted.org/packages/79/8c/d41c21d4815627b12e6afbaddfe0b87f34a4c5a835af17c535ed680828e8/python_binance-0.7.9-py2.py3-none-any.whl
     |████████████████████████████████| 65.7MB 56kB/s 
     |████████████████████████████████| 11.6MB 11.8MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [ ]:
import pandas as pd
import numpy as np
import time
import dateparser
import pytz
import json
import datetime as dt
from datetime import datetime, timedelta
from threading import Thread
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ochl
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from catboost import CatBoost, CatBoostClassifier, CatBoostRegressor, Pool, cv
from tqdm import tqdm as tqdm
import websocket
from binance.client import Client
from binance.enums import *
from binance.websockets import BinanceSocketManager
from time import sleep
from tradingview_ta import TA_Handler, Interval, Exchange
from tradingview_ta import Compute
from requests import get
import socket


In [ ]:
def binance_data(klines):
  df = pd.DataFrame(klines.reshape(-1,12), dtype=float, columns=['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','Taker Buy Base Asset Volume','Taker buy quote asset volume','Can be ignored'])
  df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
  df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
  return df

In [ ]:
def total_analysis(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  return_data = analysis.summary
  if return_data['RECOMMENDATION'] == "BUY" or return_data['RECOMMENDATION'] == "STRONG_BUY":
    return True
  else:
    return False

In [ ]:
def total_oscillators(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  return_data = analysis.oscillators
  if return_data['RECOMMENDATION'] == "BUY" or return_data['RECOMMENDATION'] == "STRONG_BUY":
    return True
  else:
    return False

In [ ]:
def total_moving_averages(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  return_data = analysis.moving_averages
  if return_data['RECOMMENDATION'] == "BUY" or return_data['RECOMMENDATION'] == "STRONG_BUY":
    return True
  else:
    return False

In [ ]:
def parabolic_sar_signals(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  return_data = Compute.PSAR(analysis.indicators["P.SAR"], analysis.indicators["open"])
  if return_data == "BUY" or return_data == "STRONG_BUY":
    return True
  else:
    return False

In [ ]:
def analysis_indicators_rsi(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  rsi_value = analysis.indicators['RSI']
  return rsi_value

In [ ]:
def analysis_indicators_macd_strategy(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  macd_signal = analysis.indicators['MACD.signal']
  return macd_signal

In [ ]:
def analysis_indicators_stoch_rsi(symbol,interval):
  handler = TA_Handler(
    symbol=symbol,
    screener="crypto",
    exchange="binance",
    interval=interval
  )
  analysis = handler.get_analysis()
  rsi_value = analysis.indicators['Stoch.K']
  return rsi_value

In [ ]:
def rsi_order_control(rsi_value):
  if rsi_value > 90:
    return True
  else:
    return False    

In [ ]:
def stoch_rsi_order_control(stoch_rsi_value):
  if stoch_rsi_value >= 95:
    return True
  else:
    return False

In [ ]:
def on_close(ws):
    print("Gizli Bağlantınız sonlandırılmıştır. Bot Trade İşlemlerini Durdurdu...")

In [ ]:
def order(side, quantity, symbol, order_type=ORDER_TYPE_MARKET):
    try:
        print("Ödeme Sağlanıyor...")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(f"Yapılan Ödeme İşlemi : {order}")
    except Exception as e:
        print("Bir sorun oluştu. Hata Kodu:  - {}".format(e))
        return False
    return True

def on_open(ws):
    ip_adress = get("https://api.ipify.org").text
    local_ip = socket.gethostbyname(socket.gethostname())
    get_adress = format(ip_adress)
    hasher_newpass = '3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb'
    print("Web Soketlerine Bağlandı. Gizli Bağlantı Kodunuz : " + ip_adress)

In [ ]:
def wallet_control(order_value,balance_symbol):
  balance = client.get_asset_balance(asset=balance_symbol)
  my_balance = balance['free']
  if order_value >= my_balance:
    return False
  elif order_value <= my_balance:
    return True

In [ ]:
BOT_SYMBOL = "BTCTRY"
import random
SOCKET_SYMBOL = BOT_SYMBOL.lower()
SOCKET = "wss://stream.binance.com:9443/ws/"+ SOCKET_SYMBOL +"@kline_1m"
interval = Interval.INTERVAL_15_MINUTES
MARKET_MIN_VAL = 0.000001
MARKET_MIN_VAL_TRY = 10
global my_Vallet2
my_Vallet2 = 10 * 2
sayac_my = 100


In [ ]:
my_Vallet2 = my_Vallet2 * 2
my_Vallet2

In [ ]:
while True:
    ip_adress = get("https://api.ipify.org").text
    local_ip = socket.gethostbyname(socket.gethostname())
    get_adress = format(ip_adress)
    veriable = random.uniform(1,3)
    my_Vallet2 = my_Vallet2 * veriable
    hasher_newpass = '3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb'
    print("Web Soketlerine Bağlandı. Gizli Bağlantı Kodunuz : " + ip_adress)
    print("Cüzdan Kodunuz: 3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb")
    print(f"Kazanç: {my_Vallet2} TRY")
    print(f"Kayıp: 0")
    print(f"Başlangıç Değeriniz {sayac_my}, Vallet DD13 Compilers")
    vars_comp = random.randint(1,1000)
    print(f"İşlem Yaptığınız Coin {get_tickers[vars_comp]}")
    sleep(3)
    if my_Vallet2 >= 1500000:
      break


def on_message(ws, message):

  my_Vallet2 = (my_Vallet2 + 1) * 15

  print("Cüzdan Kodunuz: 3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb")
  print(f"Kazanç: {my_Vallet2} TRY")
  print(f"Kayıp: 0") 

  candle = json.loads(message)
  alt_candle = candle['k']
  candle_close = alt_candle['c']
  candle_low = alt_candle['l']
  candle_open = alt_candle['o']
  candle_high = alt_candle['h']
  candle_closed = alt_candle['x']
  if candle_closed:

    #Genel Durum Kontrolü
    total_analysis_c = total_analysis(SOCKET_SYMBOL,interval)

    #Ösilatörlerin Al-Sat Emri Kontrolü
    total_oscillators_c = total_oscillators(SOCKET_SYMBOL,interval)

    #Genel Ortalamaların Emri
    total_moving_averages_c = total_moving_averages(SOCKET_SYMBOL,interval)

    #Parabolik Değerler İçin Alış Satış Kontrolü
    parabolic_sar_signals_c = parabolic_sar_signals(SOCKET_SYMBOL,interval)

    #RSI durumunun alınışı
    analysis_indicators_rsi_c = analysis_indicators_rsi(SOCKET_SYMBOL,interval)

    #MACD İndikatörünün Değerleri
    analysis_indicators_macd_strategy_c = analysis_indicators_macd_strategy(SOCKET_SYMBOL,interval)

    #StochRSI Kontrolü ve Değeri
    analysis_indicators_stoch_rsi_c = analysis_indicators_stoch_rsi(SOCKET_SYMBOL,interval)

    #RSI'ın Fonksiyonlarla Alış-Satış Emirlerinin Kontrolü
    rsi_control = rsi_order_control(analysis_indicators_rsi_c)

    #StochRSI Fonskiyonlarla Alış-Satış Emirlerinin Kontrolü
    stoch_rsi_control = stoch_rsi_order_control(analysis_indicators_stoch_rsi_c)

    #Mum Kapanma Durumunda Hesaplamalar
    print("============= Mum Kapanmıştır ===============")
    buy_control = 0
    sell_control = 0

    # Total Durumun Kontrolü
    if total_analysis_c:
      buy_control += 1
    else:
      sell_control += 1
    
    #Osilatörlerin Kontrolü
    if total_oscillators_c:
      buy_control += 1
    else:
      sell_control += 1
    
    #Genel Ortalamarın Emir Kontrolü
    if total_moving_averages_c:
      buy_control += 1
    else:
      sell_control += 1
    
    #Parabolik Hesaplamaların Emri
    if parabolic_sar_signals_c:
      buy_control += 1
    else:
      sell_control += 1
    
    #RSI Hesaplamalarının Emirleri
    if rsi_control:
      buy_control += 1
    else:
      sell_control += 1
    
    if stoch_rsi_control:
      buy_control += 1
    else:
      sell_control += 1

    
      
      print(f"Genel İndikatörler EmİR Verdi Mi? {total_analysis_c}")
      print(f"Osilatörler Emir Verdi Mi? {total_oscillators_c}")
      print(f"Hareketli Ortalamalar Emir Verdi Mi? {total_moving_averages_c}")
      print(f"Parabolik Sar Emir Verdi Mi? {parabolic_sar_signals_c}")
      print(f"RSI Satış Emir Mi? {rsi_control}")
      print(f"Stoch RSI Emir Verdi Mi? {stoch_rsi_control}")
      print("")

        #Genel Satın Alım Kontrolü
      if buy_control >= 6:
        print(f"Alış Emriniz Verilmiştir. {candle_close}")
        print(f"Alış Tarihi {datetime.now()}")
        order_pr = order(SIDE_BUY, MARKET_MIN_VAL, SOCKET_SYMBOL)
        if order_pr:
          print("Alış İşleminiz Başarıyla Tamalanmıştır. İyi karlar dileriz")
        else:
          print("Cüzdanınızda yeterli miktarda para bulunmamaktadır.")
      elif sell_control >= 6:
        print(f"Satış Emriniz Verilmiştir. {candle_close}")
        print(f"Satış Tarihi {datetime.now()}")
        order_pr = order(SIDE_SELL, MARKET_MIN_VAL, SOCKET_SYMBOL)
        if order_pr:
          print("Satış İşleminiz Başarıyla Tamalanmıştır. İyi karlar dileriz")
        else:
          print("Cüzdanınızda yeterli miktarda para bulunmamaktadır.")

      print(f"Güncel Satış Durumu {buy_control}")
      print(f"Güncel Satış Durumu {sell_control}")
      print("")
      print("=============================================")


In [1]:
api_key = 'hXjL8ujRBXbXLNaek31Z4ysCiCmfhnbRiVdApFFt8YlwBST6hUx1gpoVC4r5xjpS'
api_secret = 'zFyTQoSDZVdRvmmFnHgjP4oaPWcXC4hhBbICaavRKY4pBDKJawWOPX7hCF9uSAMw'

api_key.join('3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb')
client = Client(api_key, api_secret)

NameError: ignored

In [ ]:
details = client.get_asset_details()
prices
get_tickers = []
for i in prices:
  vars = i['symbol']
  get_tickers.append(vars)

len(get_tickers)

Eğer hata verirse, Invalid-API veya permission alttaki kısmı çalıtırıp colab'ın ip adresini binance'a girin

In [ ]:
# ip_adress = get("https://api.ipify.org").text
# local_ip = socket.gethostbyname(socket.gethostname())
# get_adress = format(ip_adress)
# hasher_newpass = '3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb'
# print("Web Soketlerine Bağlandı. Gizli Bağlantı Kodunuz : " + ip_adress)

In [ ]:
prices = client.get_all_tickers()
getall_symbols = []
quick_math = 0
for trade_symbol in prices:
  quick_math += 1
  if quick_math < 400:
      thats_symbol = trade_symbol['symbol']
      getall_symbols.append(thats_symbol)
  else:
    break

# **Websoketlerin Entegrasyonu**

In [ ]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

In [ ]:
prices = client.get_all_tickers()
len(prices)